# Building a Fraud Prediction Model with EvalML

In this demo, we will build an optimized fraud prediction model using EvalML. To optimize the model we will set up an objective function based on some assumptions about our business. 

In [1]:
%load_ext autoreload
%autoreload 2
import evalml
from evalml.objectives import FraudCost
from evalml.preprocessing import split_data

## Load Transactions

In [2]:
X, y = evalml.demos.load_fraud()

             Number of Features
Boolean                       1
Categorical                   6
Numeric                       5

Number of training examples: 99992

Labels
False    84.82%
True     15.18%
Name: fraud, dtype: object


## Define Objective

In [3]:
fraud_objective = FraudCost(
    retry_percentage=.5,
    interchange_fee=.02,
    fraud_payout_percentage=.75,
    amount_col='amount'
)

## Search for best modeling pipeline

In order to validate the results of the pipeline creation and optimization process, we will save some of our data as a holdout set

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64', 'bool']
X = X.select_dtypes(include=numerics)
X_train, X_holdout, y_train, y_holdout = split_data(X, y, test_size=.8, random_state=0)

Because the fraud labels are binary, we will use `AutoClassifier`. When we call `.fit()`, the search for the best pipeline will begin. 

In [5]:
clf = evalml.AutoClassifier(objective=fraud_objective,
                            max_time=120,
                            max_pipelines=20)

clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Fraud Cost. Lower score is better.

Searching up to 20 pipelines. Will stop searching for new pipelines after 120 seconds.

Possible model types: linear_model, xgboost, random_forest

Testing XGBoost w/ imputation: 100%|██████████| 20/20 [02:02<00:00,  6.11s/it]                     

✔ Optimization finished


### View rankings and select pipeline

Once the fitting process is done, we can see all of the pipelines that were searched, ranked by their score on the objective function we defined

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,18,XGBoostPipeline,0.007614,False,"{'eta': 0.1496748671836832, 'min_child_weight'..."
1,14,RFClassificationPipeline,0.007614,False,"{'n_estimators': 609, 'max_depth': 7, 'impute_..."
2,17,XGBoostPipeline,0.007614,False,"{'eta': 0.264555612104627, 'min_child_weight':..."
3,8,XGBoostPipeline,0.007614,False,"{'eta': 0.6481718720511973, 'min_child_weight'..."
4,16,RFClassificationPipeline,0.007614,False,"{'n_estimators': 715, 'max_depth': 7, 'impute_..."
5,0,XGBoostPipeline,0.007614,False,"{'eta': 0.5928446182250184, 'min_child_weight'..."
6,15,XGBoostPipeline,0.007614,False,"{'eta': 0.7206326547259169, 'min_child_weight'..."
7,1,XGBoostPipeline,0.007614,False,"{'eta': 0.38438170729269994, 'min_child_weight..."
8,19,XGBoostPipeline,0.007614,False,"{'eta': 0.35950790057378607, 'min_child_weight..."
9,13,XGBoostPipeline,0.007614,False,"{'eta': 0.9786183422327642, 'min_child_weight'..."


to select the best pipeline we can run

In [7]:
pipeline = clf.best_pipeline

to select another pipeline we can use the id

In [8]:
pipeline = clf.get_pipeline(0)

### Describe pipeline

You can get more details about any pipeline. Including 

In [9]:
clf.describe_pipeline(0)

************************
* Pipeline Description *
************************

Pipeline Name: XGBoost w/ imputation
Model type: xgboost
Objective: Fraud Cost (lower is better)
Total training time (including CV): 4.0 seconds

Parameters
• eta: 0.5928446182250184
• min_child_weight: 8.598391737229157
• max_depth: 4
• impute_strategy: most_frequent
• percent_features: 0.6273280598181127

Cross Validation
               F1  Precision  Recall   AUC  Log Loss  Fraud Cost # Training # Testing
0           0.264      0.152   0.264 0.856     0.188       0.008  13332.000  6666.000
1           0.264      0.152   0.264 0.844     0.190       0.008  13332.000  6666.000
2           0.264      0.152   0.264 0.836     0.198       0.008  13332.000  6666.000
mean        0.264      0.152   0.264 0.845     0.192       0.008          -         -
std         0.000      0.000   0.000 0.010     0.006       0.000          -         -
coef of var 0.000      0.000   0.000 0.012     0.030       0.002          -       